## to do 

1. rename the columns use more intuitive names
1. implement usd to euro conversion for the yahoo timeseries

In [86]:
%config Completer.use_jedi = False

In [87]:
import sys

In [88]:
sys.path.append("../streamlit_app")

In [89]:
import sys
from datetime import datetime, timedelta

import altair as alt
import numpy as np
import pandas as pd
import panel as pn
import yfinance as yf
from altair import datum
from utility import (
    plot_transactions,
    plot_transactions_2,
    read_ticker_ts,
    read_transactions,
)

from datetime import date, timedelta

In [90]:
import plotly.graph_objs as go
import plotly.express as px

In [91]:
import plotly.io as pio
pio.templates.default = 'plotly_white'

In [92]:
fln = "../data/from_2020-05-01_to_2021-04-05_MTYxODMzMTk2NzQ4MA.csv"
tr = read_transactions(fln) 

## data preparation

### keep only the relavant columns

In [93]:
col_to_keep = ['Action', 'Time', 'Ticker', 'No. of shares', 'Price / share', 'Exchange rate',
               'Result (EUR)', 'Currency (Price / share)', 'ISIN']

tr = tr[col_to_keep]

### rename columns

In [94]:
tr.loc[tr['Action'].str.contains("buy"), 'Action'] = 'buy'
tr.loc[tr['Action'].str.contains("sell"), 'Action'] = 'sell'

### feature engineering

use the transaction history to determine
- invested amount
- average price after each trasaction
- profit after each action 


In [95]:
def calculate_average_price(group):
    '''calculate the average price based on the invested value and floating number of shares'''

    # cumulative total in euro (buy price*share - average_price*sell)
    group['cum_total_eur'] = group['invested_eur'].cumsum()
    group['ave_price_eur'] = group['cum_total_eur']/group['cum_shares']

    return group


def calculate_return(group):
    """Update transaction time history, include calculation of average price
    """

    mask_buy = group["Action"] == 'buy'
    mask_sell = group["Action"] == 'sell'

    # determine the accumulated number of shares
    group.loc[mask_buy,'action_sign']= 1
    group.loc[mask_sell,'action_sign']= -1
    
    group['cum_shares'] = (group['No. of shares'] *
                           group['action_sign']).cumsum()

    # average price, treating all actions as buy
    group['pps_eur'] = group['Price / share'] / \
        group['Exchange rate'].astype('float')  # price per share in eur
    group['invested_eur'] = group['No. of shares']*group['pps_eur']
    group = calculate_average_price(group)

    # update the ave_price whenever a sell event occurs
    for idx, row in group[mask_sell].iterrows():
        group.loc[idx, 'invested_eur'] = -group.loc[idx,'No. of shares']*group.loc[idx-1, 'ave_price_eur']
        group.loc[idx, 'ave_price_eur'] = group.loc[idx-1, 'ave_price_eur']
        group = calculate_average_price(group)

    # determine the return for each sell event
    group.loc[mask_sell, 'profit_eur'] = group.loc[mask_sell, 'pps_eur']*group.loc[mask_sell, 'No. of shares'] + \
        group.loc[mask_sell, 'invested_eur']

    return group

In [96]:
tr=tr.loc[tr['Action'].isin(['buy','sell'])]
grouped = tr.groupby(by='Ticker')

groups = []  

for name , group in grouped:
    group.reset_index(inplace=True, drop=True)
    group = calculate_return(group)
    groups.append(group)
    
tr = pd.concat(groups).reset_index(drop=True)

### read ticker history from yfiance

In [97]:
tickers = tr[['Ticker', 'Currency (Price / share)', 'ISIN']].drop_duplicates()
tickers.rename({'Currency (Price / share)': 'Currency'}, axis=1, inplace=True)
# tickers = tickers.dropna(subset=['Ticker'])
us_tickers = tickers.loc[tickers['Currency']=='USD', 'Ticker'].to_list()
# us_tickers.append('USDEUR%3DX')

In [98]:
start = tr.Time.min()
end = tr.Time.max() +  timedelta(1)
data = yf.download(us_tickers, start, end)['Adj Close']

[*********************100%***********************]  44 of 44 completed

1 Failed download:
- LTM: No data found, symbol may be delisted


In [99]:
tr.Time.max()

Timestamp('2021-04-01 13:52:58')

In [100]:
df_forex = yf.download(['USDEUR%3DX'], start, end)[['Adj Close']]
df_forex = df_forex.reset_index()
df_forex.rename({'Date':"date", 'Adj Close':"rate"}, axis = 1, inplace=1)

[*********************100%***********************]  1 of 1 completed


### identify tickers not downloaded

In [101]:
mask = data.isna().mean() == 1.0

In [102]:
not_found = mask.loc[mask].keys().values

In [103]:
print(f'these tickers are not included in the time history: {not_found}')

these tickers are not included in the time history: ['LTM']


### combine yahoo time history with transaction data

In [104]:
# retrieve stock value at close
def ticker_price_history(data, ticker):
    """filter time history of the specifed ticker, history of ticker close price """
    tts_sub = data[ticker]  # ticker time series
    tts_sub = tts_sub.reset_index()
#     tts_sub.columns = tts_sub.columns.droplevel()
    tts_sub.columns = ['time_ts', 'close_price']

    return tts_sub

In [105]:
def merge_histories(tr_sub, tts_sub, ticker):
    """merge dataframe based on date and time

    note: invested amount is determined based on the market price at close, rather than the transaction record
    """
    merged = tts_sub.merge(right=tr_sub, left_on='time_ts',
                           right_on='Time', how="outer")
    merged = merged.merge(df_forex, left_on = 'time_ts', right_on='date')
    merged['cum_shares'] = merged['cum_shares'].fillna(method='ffill')
    merged['value'] = merged['close_price']*merged['cum_shares']*merged['rate']
    merged['ticker'] = ticker
    merged['cum_total_eur'] = merged['cum_total_eur'].fillna(
        method='ffill')
    merged['value'] = merged['value'].fillna(
        method='ffill')
    return merged

In [106]:
# loop it through for all the tickers:
dfs = []
groups = tr.groupby(by='Ticker')
tickers = data.columns

for ticker in tickers:
    if ticker not in not_found:
        # share_no_history(tr_pivoted,ticker)
        tr_sub = groups.get_group(
            ticker).copy()[['Time', 'Ticker', 'Action', 'No. of shares', 'cum_shares', 'cum_total_eur', 'profit_eur']]
        tr_sub['Time']=tr_sub["Time"].dt.floor("d")
        tts_sub = ticker_price_history(data, ticker)
        df = merge_histories(tr_sub, tts_sub, ticker)

        dfs.append(df)
    else:
        print(f'{ticker} not in the database')

df_combined = pd.concat(dfs)

LTM not in the database


## monthly transaction overview

Hereby I will create an overview of total transacations

In [107]:
mt = (
    tr.groupby(by=[pd.Grouper(key="Time", freq="M"), "Action"])["Action"]
    .count()
    .rename("transactions")
    .reset_index()
)  # montly transaction

In [108]:
fig = go.Figure()

# reformat the date_time to provide a monthly summary (do not show the date)
mt['mnth_yr'] = mt['Time'].apply(lambda x: x.strftime('%b-%Y'))

for action in ['buy', 'sell']:
    mt_ = mt.loc[mt['Action'] == action]
    fig.add_trace(go.Bar(
        x=mt_.mnth_yr,
        y=mt_.transactions,
        name=action,
    ))

fig.update_layout(barmode='group', xaxis_tickangle=-45, title_text='monthly transactions overview',
                  yaxis=dict(
                      title='transaction counts',
                  ))
fig.show()

## transaction over the week

In [109]:
# weekday with the most frequent transactions
# tr_ = read_transactions(fln, floor=False)
day_names = tr.Time.dt.day_name()

# time of the day when most likely to trade
hours = tr.Time.round('1h').dt.time
hours.value_counts().idxmax()

datetime.time(15, 0)

In [110]:
from plotly.subplots import make_subplots

fig = go.Figure()

fig = make_subplots(rows=1, cols=2, subplot_titles=[
                    'distribution over the day', 'transactions distribution over the week'])

fig.add_trace(go.Histogram(
    x=sorted(hours),
), row=1, col=1)

m = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
fig.add_trace(go.Histogram(
    x=sorted(day_names, key=m.index),
), row=1, col=2)

fig.update_xaxes(title_text='time of day', row=1, col=1, tickformat="%H:%M")
fig.update_yaxes(title_text='counts', row=1, col=1)


fig.update_xaxes(
    title_text='day of a week',
    row=1, col=2,
    tickformat="%H-%M-%S"
)
fig.update_yaxes(title_text='counts', row=1, col=2)

fig.update_layout(bargap=0.2)
fig.update_layout(showlegend=False) 
fig.show()

In [111]:
print(f"You are more likely to trade on {day_names.value_counts().idxmax()} than the other weekdays. When looking at the distribution over the day, {hours.value_counts().idxmax():%H} o'clock is the peak hour for you to place an order.")

You are more likely to trade on Thursday than the other weekdays. When looking at the distribution over the day, 15 o'clock is the peak hour for you to place an order.


## latest portofolio

1. what are the stocks in my pf?
1. total value (weekly overview)

### pie chart of portofolio composition

In [112]:
df_combined

,time_ts,close_price,Time,Ticker,Action,No. of shares,cum_shares,cum_total_eur,profit_eur,date,rate,value,ticker
0,2020-05-01,10.640000,NaT,NaN,NaN,NaN,NaN,NaN,NaN,2020-05-01,0.913620,NaN,AAL
1,2020-05-04,9.820000,NaT,NaN,NaN,NaN,NaN,NaN,NaN,2020-05-04,0.912440,NaN,AAL
2,2020-05-05,9.510000,NaT,NaN,NaN,NaN,NaN,NaN,NaN,2020-05-05,0.917070,NaN,AAL
3,2020-05-06,9.250000,NaT,NaN,NaN,NaN,NaN,NaN,NaN,2020-05-06,0.922630,NaN,AAL
4,2020-05-07,9.540000,NaT,NaN,NaN,NaN,NaN,NaN,NaN,2020-05-07,0.926050,NaN,AAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,2021-03-26,319.950012,NaT,NaN,NaN,NaN,1.75,540.54122,NaN,2021-03-26,0.849140,475.444112,ZM
228,2021-03-29,308.390015,NaT,NaN,NaN,NaN,1.75,540.54122,NaN,2021-03-29,0.848320,457.823484,ZM
229,2021-03-30,313.190002,NaT,NaN,NaN,NaN,1.75,540.54122,NaN,2021-03-30,0.849545,465.620752,ZM
230,2021-03-31,321.290009,NaT,NaN,NaN,NaN,1.75,540.54122,NaN,2021-03-31,0.852980,479.594425,ZM


In [113]:
(end - timedelta(1))

Timestamp('2021-04-01 13:52:58')

In [114]:
df_ = df_combined.loc[(df_combined['time_ts'] == (end -
                           timedelta(1)).floor('1d')) 
                      & (df_combined['cum_shares'] > 0.25)
                     ]
df_ = df_.drop_duplicates(
    subset=['time_ts', 'ticker'], keep='last', inplace=False, ignore_index=False)

fig22 = px.pie(df_, values='value', names='ticker',
               title='portofolio composition')

In [115]:
fig22

### line plot for portofolio ts

In [116]:
df_combined = df_combined.drop_duplicates(
    subset=['time_ts', 'ticker'], keep='last', inplace=False, ignore_index=False)

In [117]:
df_agg = df_combined.pivot_table(index='time_ts', values=[
                                 'value', 'cum_total_eur', 'profit_eur'], aggfunc='sum').reset_index()

In [118]:
df_agg=df_agg.rename({"time_ts":'time',
              'value':'open position value',
              'cum_total_eur':'invested amount'}, axis =1)
df_agg['realized profit'] = df_agg['profit_eur'].cumsum()
df_agg['floating profit'] = df_agg['open position value'] - df_agg['invested amount']

In [119]:

# df_agg[]
fig = px.line(df_agg, x="time", y=['open position value','invested amount'],
              hover_data={"time": "|%B %d, %Y"},
              title='Overview',
              )

fig.data[0].name = 'open positions'
fig.data[1].name = 'invested amount'
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")

fig.update_layout(hovermode="x", )
fig.update_layout(legend_title_text= 'total value of:') 
fig.update_yaxes(title='EUR')
fig.update_traces(showlegend=True)

fig.show()

### brief summary

In [120]:
last_row = df_agg.tail(1)

In [121]:
# fig.data

In [122]:
print('A brief summary')
print("You started investing with Trading 212 on {:}.".format(
    start.strftime("%b %d, %Y")))
print("Upon {:}, you've invested {:0.2f} EUR in the market, with a floating profit of {:0.2f} EUR. Total realized profit amounts {:0.2f} EUR".format(
    end.strftime("%b %d, %Y"), 
    last_row['invested amount'].values[0],
    last_row["floating profit"].values[0],
    last_row["realized profit"].values[0]))

A brief summary
You started investing with Trading 212 on May 01, 2020.
Upon Apr 02, 2021, you've invested 13965.02 EUR in the market, with a floating profit of -691.40 EUR. Total realized profit amounts 906.82 EUR


### line plot of profit and loss

In [123]:

fig = px.line(df_agg, x="time", y=['floating profit','realized profit'],
              hover_data={"time": "|%B %d, %Y"},
              title='Overview'
              )


fig.add_trace(go.Scatter(
    name="total",
    x=df_agg["time"], y=df_agg['floating profit'] + df_agg['realized profit'],
))

fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")

fig.update_layout(hovermode="x")

fig.show()

## stock analysis

### correlation analysis

In [133]:
from matplotlib import pyplot as plt
cols = df_.ticker.dropna().unique()
retscomp = data.loc[:,  cols]
retscomp = retscomp.pct_change()
corr = retscomp.corr()

In [134]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Heatmap(
    z=corr,
    x=cols,
    y=cols,
    colorscale='Hot',
    reversescale=True,
    zmax=1.0,
    zmin=0.0))

fig.update_layout(
    autosize=False,
    width=600,
    height=600,)

fig.show()

SQ BILI


### Stocks mean and Risk calculation

In [126]:
y=retscomp.std()

In [155]:

fig = px.scatter(x=retscomp.mean(), y=retscomp.std(), text=cols)
fig.update_traces(textposition='top center')
fig.update_layout(
#     height=800,
    title_text='Return and Risk',
    xaxis_title="Return",
    yaxis_title="Risk",
)
fig.show()

'BIDU'

## performance of each stock over time 

In [128]:
pd.options.plotting.backend = "plotly" 

In [129]:
# get the tickers with the largest share in the portofolio 
df_end_sorted = df_.sort_values(by='value', ascending=False)
tickers = df_end_sorted.iloc[0:5]['ticker'].values

In [130]:
def pct_change(x): 
    x_ = x.fillna(method='bfill')
    return x_.div(x_.iloc[0]).subtract(1)

In [131]:
# slice the selected tickers 
# df_combined['close_price_pct_change'] = df_combined['close_price'].pct_change()
ticker_mask = df_combined.ticker.isin(tickers)
df_combined['pct_change'] = df_combined.groupby('ticker')['close_price'].apply(
    lambda x: pct_change(x))

df_ = df_combined.loc[ticker_mask]

<ipython-input-131-de95b38ff07c>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [132]:
# symbol price history
fig = px.line(df_combined, x="time_ts",
              y="pct_change", color='ticker')

# add my transaction records
fig.add_trace(px.scatter(df_combined.dropna(
    subset=['Action']), x="time_ts", y="pct_change", color='Action').data[0])

fig.show()